In [1]:
# Compare three different algorithms: Dynamic Pricing, Adaptive Pricing, and FCFS
# Use intercepts and slopes from initialization.py as starting point for linear demand curve
# Dynamic Pricing: 
    # Retail Price Optimization at InterContinental Hotels Group. 
    # INFORMS Journal on Applied Analytics 42(1):45-57. 
    # https://doi.org/10.1287/inte.1110.0620

# Adaptibe Pricing: Developed by me, adapted from:
    # Revenue Management Without Forecasting or Optimization: An Adaptive Algorithm for Determining Airline Seat Protection Levels
    # Management Science 46(6):760-775.
    # https://doi.org/10.1287/mnsc.46.6.760.11936
    
# FCFS: First-Come, First-Serve

In [2]:
import itertools
from operator import itemgetter
import numpy as np
from scipy.stats import norm
from scipy.stats import poisson
from cvxopt import matrix, solvers, spmatrix

# Import linear demand curve coefficients and initialized paratemers 
from initialize import initialize
from genregparam import linparams
from RADs import stay_index

np.set_printoptions(precision=2, suppress=True)

In [3]:
# Booking classes for each stay night for week 1
# There is no Saturday or Friday arrivals that span Sunday night if it is week 1
# Similarly, there is no Saturday arrivals that span Monday night in week 1
stay_index_wk1 = [(0, 0, 0), (0, 0, 1), (0, 0, 2), (1, 0, 0), (1, 0, 1), (1, 0, 2)]
mon_stay_index_wk1 = [(0, 1, 0), (0, 1, 1), (0, 1, 2), (0, 0, 1), (0, 0, 2), 
                     (1, 1, 0), (1, 1, 1), (1, 1, 2), (1, 0, 1), (1, 0, 2)]
stay_index_wk1 = [stay_index_wk1, mon_stay_index_wk1] + stay_index[2:]

In [4]:
# Parameters for simulation study
weeks = 52
n_class = 2
los = 3
capacity = 100
intensity = 1.5
slopes_init = np.array([-0.1, -0.15])
rates_init = np.array([[135, 135, 135, 135, 135, 108, 108],
                       [115, 115, 115, 115, 115, 92, 92]])

# For adaptive pricing algorithm, we have two parameters for step size: param1 and param2
param1, param2 = 200, 10

# Total combinations of arrivals
combs = n_class * 7 * los

In [5]:
# Partitioned protection levels, nested protection levels, representative revenue, and
# discount ration for each virtual bucket, each stay night
buckets, thetas_prt, thetas, rates_vir, ratios = initialize(capacity, intensity, rates_init)
slopes, intercepts = linparams(capacity, intensity, slopes_init, rates_init)

In [6]:
thetas_old = [np.minimum(thetas[i], capacity)[:-1] for i in range(7)]
thetas_old

[array([ 12.,  36., 100.]),
 array([12., 36., 72., 96.]),
 array([12., 24., 72., 84.]),
 array([36., 60., 72.]),
 array([24., 60., 72.]),
 array([12., 24., 36., 60.]),
 array([12., 24., 48., 72.])]

In [7]:
# Define a function to calculate rates for multiple stay nights
def moving_sum(a, n):
    ret = np.cumsum(a, dtype=float)
    ret[n:] = ret[n:] - ret[:-n]
    return ret[n-1:]

# Define a function to compute sum of first half and second half of an array
def sum_of_elements(arr):
    n = len(arr) // 2
    sumfirst = sum(arr[:n])
    sumsecond = sum(arr[n:])
    return sumfirst, sumsecond

In [8]:
# List all the buckets that a booking class belongs to
# These buckets could span multiple stay nights
bkClass_bkt = []
for i in range(len(buckets)):
    for j in range(len(buckets[i])):
        for item in buckets[i][j]:
            bkClass_bkt.append((item, (i, j)))

In [9]:
############################ Quadratic Programming ############################

############################ Step 1 ############################
# Flatten arrays for quadratic programming formulation
slopes_flat = slopes.reshape(n_class * 7 * los)
intercepts_flat = intercepts.reshape(n_class * 7 * los)

############################ Step 2 ############################
# Inequality equations, LHS
# We have total number of 42 decision veriables, corresponding to total number of
# rate class, arrival day of week and los combinations.
# Column indexes 0-20 are associated with decision variables for rate class 1
# Column indexes 21-41 are associated with decision variables for rate class 2
G = np.zeros(7 * los * n_class * 7).reshape(7, n_class*7*los)
# Arrivals that span Sunday stay night for rate class 1
G[0,:(7*los)] = [1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
# Arrivals that span Monday stay night for rate class 1
G[1,:(7*los)] = [0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
G[2,:(7*los)] = [0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
G[3,:(7*los)] = [0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
G[4,:(7*los)] = [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0]
G[5,:(7*los)] = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0]
G[6,:(7*los)] = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1]
# Arrivals that span Sunday stay night for rate class 2
G[0,(7*los):] = [1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
# Arrivals that span Monday stay night for rate class 2
G[1,(7*los):] = [0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
G[2,(7*los):] = [0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
G[3,(7*los):] = [0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
G[4,(7*los):] = [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0]
G[5,(7*los):] = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0]
G[6,(7*los):] = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1]

h1 = intercepts_flat * G

############################ Step 3 ############################
# Here, be careful. For the capacity constraints, LHS is sum of demands that span
# a specific stay night in question. But decision variables for quadratic programming
# are rates. 
# demand1 + demand2 + demand3 = intercept1 + slope1 * rate1 + intercept2 + slope2 * rate2 + 
# intercept3 + slope3 * rate3 <= capacity
# --> slope1*rate1 + slope2*rate2 + slope3*rate3 <= capacity - (intercept1+intercept2+intercept3)
h1 = np.sum(h1, axis=1)
# G for capacity constraints, Negative identity matrix for non-negativity
G = slopes_flat * G
G = np.concatenate((G, -np.identity(combs)), axis=0)
# Inequality equations, RHS
# First h for capacity rhs and second component for non-negativity rhs.
h = np.concatenate((capacity * np.ones(7) - h1, np.zeros(combs)), axis=0)

############################ Step 4 ############################
# This part is a little bit tedious, but I couldn't find an elegant way of doing it
# Purpose of this section is to make sure optimal rates for e.g., Monday arrival two-night 
# stay is equal to optimal rate for Monday arrival one-night stay and Tuesday arrival one-
# night stay. This is how the rates are calculated in hotel industry for multiple nights stay.
# It is different from airline pricing with multiple legs.
arr1 = [3*i for i in range(14)]
arr2 = [(3*i) for i in range(1, 7)] + [0] + [(3*i) % 42 for i in range(8, 14)] + [21]
arr3 = [3*(i+2) % 21 for i in range(7)] + [3*(i+2) for i in range(7, 12)] + [21] + [24]
arr4 = np.concatenate((np.arange(1, 41, 3), np.arange(2, 42, 3))).tolist()
els = np.concatenate((np.repeat(1.0, 70), np.repeat(-1, 28)))
A = spmatrix(els, np.concatenate((range(28), range(28), range(14, 28), range(28))).tolist(), 
             arr1 + arr1 + arr2 + arr2 + arr3 + arr4)
b = matrix(np.zeros(28))

############################ Step 5 ############################
# Quadratic programming
#                  minimize (1/2)x_T*Q*x + p_T*x
#                  subject to G*x <= h
#                             A*x = b
slopes_diag = np.diag(slopes_flat)
Q = 2 * matrix(-slopes_diag)
p = matrix(-intercepts_flat)

# Convert numpy arrays to cvxopt matrix forms
G = matrix(G)
h = matrix(h)

############################ Step 6 ############################
# Solve quadratic programming
sol = solvers.qp(Q, p, G, h, A, b)
print(sol['status'])

############################ Step 7 ############################
rates_DP = np.array(sol['x']).reshape(n_class, 7, los)
rates_DP = np.round(rates_DP, 0)

############################ Step 8 ############################
# Check if the rates for multiple los is satisfied, one can look at rate for e.g., Sunday arrival
# three night stay for rate class 1, $405, should equal to Sunday arrival one-night stay,
# Monday arrival one-night stay, and Tuesday arrival, one-night stay.
# And indeed 405 = 112 + 149 + 143 (exits rounding error, of course)
rates_DP

optimal


array([[[ 90., 254., 399.],
        [164., 309., 459.],
        [145., 295., 436.],
        [150., 291., 421.],
        [140., 271., 421.],
        [130., 280., 371.],
        [150., 240., 404.]],

       [[ 70., 219., 353.],
        [149., 284., 414.],
        [134., 265., 390.],
        [130., 256., 365.],
        [126., 235., 375.],
        [109., 249., 319.],
        [140., 210., 359.]]])

In [10]:
######################## Start Simulation ########################
################################################################################################
################################################################################################
################################################################################################
################################################################################################
################################################################################################

In [30]:
######################## Week 1 ########################

######################## Step1 : Based on the rates, generate demand ########################
# Initialize rates for adaptive pricing algorithm and first-come, first-serve approach
# In current week, store rates for current week and next week
wk1_rates_AP = rates_init
wk2_rates_AP = rates_init
rates_for_two_weeks_AP = np.concatenate((wk1_rates_AP, wk2_rates_AP)).reshape(2, n_class, 7)
# Derive rates for rate class of r, arrival day of d and length of stay of d
rates_AP = [[moving_sum(rates_for_two_weeks_AP[:, i].reshape(2*7), j)[:7] 
                     for j in range(1, los+1)] for i in range(n_class)]
# Convert to the right shape for later calculation
rates_AP = np.array(rates_AP)
rates_AP = np.swapaxes(rates_AP, 1, 2)

# In current week, store rates for current week and next week
wk1_rates_FCFS = rates_init
wk2_rates_FCFS = rates_init
rates_for_two_weeks_FCFS = np.concatenate((wk1_rates_FCFS, wk2_rates_FCFS)).reshape(2, n_class, 7)
# Derive rates for rate class of r, arrival day of d and length of stay of d
rates_FCFS = [[moving_sum(rates_for_two_weeks_FCFS[:, i].reshape(2*7), j)[:7] 
                       for j in range(1, los+1)] for i in range(n_class)]
# Convert to the right shape for later calculation
rates_FCFS = np.array(rates_FCFS)
rates_FCFS = np.swapaxes(rates_FCFS, 1, 2)

# Order of rates as dynamic pricing rates, adaptive pricing rates, and FCFS rates
ratesAll = [rates_DP, rates_AP, rates_FCFS]

# Generate nonhomogenous Poisson process true bookings (i.e., customer demand) on a weekly basis
# the mean of the Poisson process will be equal to the linear demand curve function of the room rates
mus = [np.maximum(intercepts + slopes * rates, 0) for rates in ratesAll]

demands = [[poisson.rvs(mu, size=1) for mu in np.nditer(mus_each)] for mus_each in mus]
demands = [np.array(x).reshape(n_class, 7, los) for x in demands]

demands_old_FCFS = demands[2]
demands_new_FCFS = demands[2]

######################## Step 2: Calculate weekly room revenue ########################
# Consider first week stay nights, our week starts on Sunday, not Monday
nightlyRev = []
wk1SellInfo = []
wk1RoomSold_AP = []
for i in range(7):
    # Sunday of first week, there is no previous week Saturday or Friday arrival that span Sunday
    if i == 0:
        buckets_night = [[rad for rad in rads if rad[1] == 0] for rads in buckets[i]]
    # Monday of first week, there is no previous week Saturday arrival that span Monday
    elif i == 1:
        buckets_night = [[rad for rad in rads if rad[1] != 6] for rads in buckets[i]]
    else:
        buckets_night = buckets[i]
    # Three different algorithms, three different scenarios to consider
    capacity_left = [capacity] * 3
    # Info about the booking class type, rooms sold and revenues (single night revenue)
    soldAll = []
    roomSold_AP = []
    # Reverse the buckets for each night under the assumption that low ranked booking
    # classes arrive first
    pl_idx = 0
    for rads in reversed(buckets_night):
        sold_bucket = []
        sold_bucket_AP = 0
        for rad in rads:
            # For stay night revenue calculation, we only use one-night stay revenue, not
            # multiple stay night total revenue
            rad_single = (rad[0], i, 0)
            # Rooms sold equals smaller of demand and capacity left
            # Dynamic pricing
            sold_DP = min(demands[0][rad], capacity_left[0])
            # Adaptive pricing, the selling amount is constrained by the protection levels for higher classes
            try:
                BL_AP = max(capacity_left[1]-list(reversed(thetas_old[i]))[pl_idx], 0)
            except IndexError:
                BL_AP = capacity_left[1]
            
            sold_AP = min(demands[1][rad], BL_AP)
            sold_bucket_AP += sold_AP
            # First-come, first-serve
            sold_FCFS = min(demands[2][rad], capacity_left[2])
            
            sold = [sold_DP, sold_AP, sold_FCFS]
            
            rev = [ratesAll[i][rad_single] * sold[i] for i in range(len(demands))]
            sold_bucket.append((rad, sold, rev))
            # Update remaining capacity for the next virtual class
            capacity_left = [capacity_left[i]-sold[i] for i in range(len(sold))]
        soldAll.append(sold_bucket)
        roomSold_AP.append(sold_bucket_AP)
        pl_idx += 1

    # Remove empty lists
    soldAll = list(filter(None, soldAll))
    soldAll = list(itertools.chain.from_iterable(soldAll))
    wk1SellInfo.append(soldAll)
    wk1RoomSold_AP.append(roomSold_AP)

    # Extract revenue information and store it in revenue array
    revenue = [soldAll[i][2] for i in range(len(soldAll))]
    nightlyRev.append(revenue)
    
    # Calculate weekly revenue for each algorithm
nightlyRev = [np.array(x) for x in nightlyRev]
revSum = [np.sum(x, axis=0) for x in nightlyRev]
wk1Rev = np.sum(revSum, axis=0)

######################## Step 3: Update protection levels ########################
# Use adaptive pricing algoritm to derive new rates for week 3
wk1RoomSold_AP = [list(reversed(wk1RoomSold_AP[i])) for i in range(len(wk1RoomSold_AP))]
wk1RoomSold_AP = [np.array(x) for x in wk1RoomSold_AP]

# Update protection levels according to the sales info
roomSold_cumsum = [np.cumsum(x) for x in wk1RoomSold_AP]

# Compute if the demand for a class exceeds its corresponding protection levels
Y = [roomSold_cumsum[i][:-1] >= thetas_old[i] for i in range(7)]
# Implement Equation(2) in vanRyzin-McGill 2000
Z = [np.cumproduct(Y[i]) for i in range(7)]
# Calculate H(theta, x)
H = [ratios[i][1:] - Z[i] for i in range(7)]
thetas_new = [thetas_old[i] - (param1/(param2+1)) * H[i] for i in range(7)]
# Truncate at zero and sort it-- nonnegativity of protection levels 
thetas_new = [np.minimum(np.maximum(thetas_new[i], 0), capacity) for i in range(7)]
thetas_new = [sorted(thetas_new[i]) for i in range(7)]
# Round to integers
thetas_new = [np.round(thetas_new[i], 0) for i in range(7)]

######################## Step 4: Derive protection level adjustments ########################
# Create a dummy booking class 1 so that we can find the partitioned protection levels
# from nested ones in an easy way
thetas_old_full = [np.concatenate(([0], thetas_old[i], [capacity])) for i in range(7)]
thetas_new_full = [np.concatenate(([0], thetas_new[i], [capacity])) for i in range(7)]

# Calculate partitioned protection level changes for each bucket in each night
thetas_old_full_prt = [np.diff(thetas_old_full[i]) for i in range(7)]
thetas_new_full_prt = [np.diff(thetas_new_full[i]) for i in range(7)]

# Percent change for partitioned protection levels
# When divide by zero, we assume the change is 1, or 100%.
thetas_adj = [np.divide(thetas_new_full_prt[i], thetas_old_full_prt[i], 
                        out=(np.zeros_like(thetas_new_full_prt[i])+2),
                       where=thetas_old_full_prt[i]!=0) - 1 for i in range(7)]

######################## Step 4: Derive average rate adjustments ########################
# Compute all the changes to booking classes across all stay nights of the week
bkClass_bkt_uniq = {}
rates_adj = {}
for x, y in bkClass_bkt:
    if x in bkClass_bkt_uniq:
        bkClass_bkt_uniq[x].append((y))
        rates_adj[x].append((thetas_adj[y[0]][y[1]]))
    else:
        bkClass_bkt_uniq[x] = [(y)]
        rates_adj[x] = [(thetas_adj[y[0]][y[1]])]
        
# Derive average changes for a booking class (i.e., r, a, d combination)
rates_adj_avg = {}
for k, v in rates_adj.items():
    avg_adj = np.round(np.mean(np.array(v)), 4)
    rates_adj_avg[k] = [(avg_adj)]
    single_rate = np.round((rates_AP[k] * (1+rates_adj_avg[k][0])) / (k[2] + 1), 0)
    rates_adj_avg[k].append((single_rate))
    
######################## Step 5: Derive single stay night rate recommendation ########################    
# Derive single stay night revenue for each rate class for each stay night of the week
rate0_new = []
rate1_new = []
for bkt in buckets:
    # Flatten the bucket elements for each day of the week
    bkt_ls = list(itertools.chain.from_iterable(bkt))
    # For each booking class in the falttened list, extract the rates generated by the algorithm
    myRate0_dict = {my_key: rates_adj_avg[my_key] for my_key in bkt_ls if my_key[0] == 0}
    myRate1_dict = {my_key: rates_adj_avg[my_key] for my_key in bkt_ls if my_key[0] == 1}
    # Store it as an array
    myRate0 = np.array(list(myRate0_dict.values()))
    myRate1 = np.array(list(myRate1_dict.values()))
    # Calculate stay night rates for each rate class for each stay night
    myRate0_avg = np.round(np.mean(myRate0, axis=0)[1], 0)
    myRate1_avg = np.round(np.mean(myRate1, axis=0)[1], 0)
    # Create list for new rates to use for next week
    rate0_new.append(myRate0_avg)
    rate1_new.append(myRate1_avg)

# New single night rates for next week
rates_update_AP = np.concatenate((rate0_new, rate1_new)).reshape(n_class, 7)
rates_update_AP

array([[103., 152., 126., 117., 114., 117., 110.],
       [124., 196., 191., 176., 152., 128., 128.]])

In [27]:
# Consider first week stay nights, our week starts on Sunday, not Monday
nightlyRev = []
wk1SellInfo = []
wk1RoomSold_AP = []
for i in range(7):
    # Sunday of first week, there is no previous week Saturday or Friday arrival that span Sunday
    if i == 0:
        buckets_night = [[rad for rad in rads if rad[1] == 0] for rads in buckets[i]]
    # Monday of first week, there is no previous week Saturday arrival that span Monday
    elif i == 1:
        buckets_night = [[rad for rad in rads if rad[1] != 6] for rads in buckets[i]]
    else:
        buckets_night = buckets[i]
    # Three different algorithms, three different scenarios to consider
    capacity_left = [capacity] * 3
    # Info about the booking class type, rooms sold and revenues (single night revenue)
    soldAll = []
    roomSold_AP = []
    # Reverse the buckets for each night under the assumption that low ranked booking
    # classes arrive first
    pl_idx = 0
    for rads in reversed(buckets_night):
        sold_bucket = []
        sold_bucket_AP = 0
        for rad in rads:
            # For stay night revenue calculation, we only use one-night stay revenue, not
            # multiple stay night total revenue
            rad_single = (rad[0], i, 0)
            # Rooms sold equals smaller of demand and capacity left
            # Dynamic pricing
            sold_DP = min(demands[0][rad], capacity_left[0])
            # Adaptive pricing, the selling amount is constrained by the protection levels for higher classes
            try:
                BL_AP = max(capacity_left[1]-list(reversed(thetas_old[i]))[pl_idx], 0)
            except IndexError:
                BL_AP = capacity_left[1]
            
            sold_AP = min(demands[1][rad], BL_AP)
            sold_bucket_AP += sold_AP
            # First-come, first-serve
            sold_FCFS = min(demands[2][rad], capacity_left[2])
            
            sold = [sold_DP, sold_AP, sold_FCFS]
            
            rev = [ratesAll[i][rad_single] * sold[i] for i in range(len(demands))]
            sold_bucket.append((rad, sold, rev))
            # Update remaining capacity for the next virtual class
            capacity_left = [capacity_left[i]-sold[i] for i in range(len(sold))]
        soldAll.append(sold_bucket)
        roomSold_AP.append(sold_bucket_AP)
        pl_idx += 1

    # Remove empty lists
    soldAll = list(filter(None, soldAll))
    soldAll = list(itertools.chain.from_iterable(soldAll))
    wk1SellInfo.append(soldAll)
    wk1RoomSold_AP.append(roomSold_AP)

    # Extract revenue information and store it in revenue array
    revenue = [soldAll[i][2] for i in range(len(soldAll))]
    nightlyRev.append(revenue)
    
    # Calculate weekly revenue for each algorithm
nightlyRev = [np.array(x) for x in nightlyRev]
revSum = [np.sum(x, axis=0) for x in nightlyRev]
wk1Rev = np.sum(revSum, axis=0)



array([89268., 67908., 79611.])

In [14]:
# Use adaptive pricing algoritm to derive new rates for week 3
wk1RoomSold_AP = [list(reversed(wk1RoomSold_AP[i])) for i in range(len(wk1RoomSold_AP))]
wk1RoomSold_AP = [np.array(x) for x in wk1RoomSold_AP]

# Update protection levels according to the sales info
roomSold_cumsum = [np.cumsum(x) for x in wk1RoomSold_AP]

In [15]:
# Compute if the demand for a class exceeds its corresponding protection levels
Y = [roomSold_cumsum[i][:-1] >= thetas_old[i] for i in range(7)]
# Implement Equation(2) in vanRyzin-McGill 2000
Z = [np.cumproduct(Y[i]) for i in range(7)]
# Calculate H(theta, x)
H = [ratios[i][1:] - Z[i] for i in range(7)]
thetas_new = [thetas_old[i] - (param1/(param2+1)) * H[i] for i in range(7)]
# Truncate at zero and sort it-- nonnegativity of protection levels 
thetas_new = [np.minimum(np.maximum(thetas_new[i], 0), capacity) for i in range(7)]
thetas_new = [sorted(thetas_new[i]) for i in range(7)]
# Round to integers
thetas_new = [np.round(thetas_new[i], 0) for i in range(7)]
thetas_old, thetas_new

([array([ 12.,  36., 100.]),
  array([12., 36., 72., 96.]),
  array([12., 24., 72., 84.]),
  array([36., 60., 72.]),
  array([24., 60., 72.]),
  array([12., 24., 36., 60.]),
  array([12., 24., 48., 72.])],
 [array([ 0., 26., 90.]),
  array([ 0., 23., 59., 88.]),
  array([ 0., 11., 61., 74.]),
  array([23., 47., 60.]),
  array([11., 47., 60.]),
  array([ 9., 14., 25., 52.]),
  array([ 0., 11., 37., 63.])])

In [16]:
# Create a dummy booking class 1 so that we can find the partitioned protection levels
# from nested ones in an easy way
thetas_old_full = [np.concatenate(([0], thetas_old[i], [capacity])) for i in range(7)]
thetas_new_full = [np.concatenate(([0], thetas_new[i], [capacity])) for i in range(7)]

# Calculate partitioned protection level changes for each bucket in each night
thetas_old_full_prt = [np.diff(thetas_old_full[i]) for i in range(7)]
thetas_new_full_prt = [np.diff(thetas_new_full[i]) for i in range(7)]

# Percent change for partitioned protection levels
# When divide by zero, we assume the change is 1, or 100%.
thetas_adj = [np.divide(thetas_new_full_prt[i], thetas_old_full_prt[i], 
                        out=(np.zeros_like(thetas_new_full_prt[i])+2),
                       where=thetas_old_full_prt[i]!=0) - 1 for i in range(7)]

In [17]:
thetas_old_full_prt, thetas_new_full_prt

([array([12., 24., 64.,  0.]),
  array([12., 24., 36., 24.,  4.]),
  array([12., 12., 48., 12., 16.]),
  array([36., 24., 12., 28.]),
  array([24., 36., 12., 28.]),
  array([12., 12., 12., 24., 40.]),
  array([12., 12., 24., 24., 28.])],
 [array([ 0., 26., 64., 10.]),
  array([ 0., 23., 36., 29., 12.]),
  array([ 0., 11., 50., 13., 26.]),
  array([23., 24., 13., 40.]),
  array([11., 36., 13., 40.]),
  array([ 9.,  5., 11., 27., 48.]),
  array([ 0., 11., 26., 26., 37.])])

In [18]:
thetas_adj

[array([-1.  ,  0.08,  0.  ,  1.  ]),
 array([-1.  , -0.04,  0.  ,  0.21,  2.  ]),
 array([-1.  , -0.08,  0.04,  0.08,  0.62]),
 array([-0.36,  0.  ,  0.08,  0.43]),
 array([-0.54,  0.  ,  0.08,  0.43]),
 array([-0.25, -0.58, -0.08,  0.12,  0.2 ]),
 array([-1.  , -0.08,  0.08,  0.08,  0.32])]

In [19]:
bkClass_bkt_uniq = {}
rates_adj = {}
for x, y in bkClass_bkt:
    if x in bkClass_bkt_uniq:
        bkClass_bkt_uniq[x].append((y))
        rates_adj[x].append((thetas_adj[y[0]][y[1]]))
    else:
        bkClass_bkt_uniq[x] = [(y)]
        rates_adj[x] = [(thetas_adj[y[0]][y[1]])]

In [20]:
# Derive average changes for a booking class (i.e., r, a, d combination)
rates_adj_avg = {}
for k, v in rates_adj.items():
    avg_adj = np.round(np.mean(np.array(v)), 4)
    rates_adj_avg[k] = [(avg_adj)]
    single_rate = np.round((rates_rad_old_AP[k] * (1+rates_adj_avg[k][0])) / (k[2] + 1), 0)
    rates_adj_avg[k].append((single_rate))

In [21]:
# Derive single stay night revenue for each rate class for each stay night of the week
rate0_new = []
rate1_new = []
for bkt in buckets:
    # Flatten the bucket elements for each day of the week
    bkt_ls = list(itertools.chain.from_iterable(bkt))
    # For each booking class in the falttened list, extract the rates generated by the algorithm
    myRate0_dict = {my_key: rates_adj_avg[my_key] for my_key in bkt_ls if my_key[0] == 0}
    myRate1_dict = {my_key: rates_adj_avg[my_key] for my_key in bkt_ls if my_key[0] == 1}
    # Store it as an array
    myRate0 = np.array(list(myRate0_dict.values()))
    myRate1 = np.array(list(myRate1_dict.values()))
    # Calculate stay night rates for each rate class for each stay night
    myRate0_avg = np.round(np.mean(myRate0, axis=0)[1], 0)
    myRate1_avg = np.round(np.mean(myRate1, axis=0)[1], 0)
    # Create list for new rates to use for next week
    rate0_new.append(myRate0_avg)
    rate1_new.append(myRate1_avg)

# New single night rates for next week
rates_update_AP = np.concatenate((rate0_new, rate1_new)).reshape(n_class, 7)

In [22]:
rates_update_AP

array([[100., 152., 126., 122., 121., 121., 108.],
       [121., 196., 191., 176., 152., 126., 125.]])

In [32]:
# Compute total demand by rate class for each stay night, which is then 
# used to calculate demand change ratio for FCFS strategy
demands_old_bynight_FCFS = [sum_of_elements(demands_old_FCFS[tuple(zip(*stay_index_wk1[i]))]) 
                            for i in range(7)]
demands_new_bynight_FCFS = [sum_of_elements(demands_new_FCFS[tuple(zip(*stay_index_wk1[i]))]) 
                            for i in range(7)]

demands_old_bynight_FCFS = np.array(demands_old_bynight_FCFS)
demands_new_bynight_FCFS = np.array(demands_new_bynight_FCFS)
with np.errstate(divide='ignore'):
    demands_adj_ratio = (demands_new_bynight_FCFS - demands_old_bynight_FCFS) / demands_old_bynight_FCFS
demands_adj_ratio[np.isnan(demands_adj_ratio)] = 1

demands_adj_ratio = np.swapaxes(demands_adj_ratio, 0, 1)
demands_adj_ratio = np.maximum(-1, np.minimum(demands_adj_ratio, 1))
rates_update_FCFS = wk1_rates_FCFS * (1 + demands_adj_ratio)

In [33]:
######################## Week 2 ########################
weeklyRev = []
for week in range(2, weeks+1):
    # First of all, protection levels are updated
    thetas_old = thetas_new
    demands_old_FCFS = demands_new_FCFS
    # In current week, store rates for current week and next week
    wk1_rates_AP = wk2_rates_AP
    wk2_rates_AP = rates_update_AP
    rates_for_two_weeks_AP = np.concatenate((wk1_rates_AP, wk2_rates_AP)).reshape(2, n_class, 7)
    # Derive rates for rate class of r, arrival day of d and length of stay of d
    rates_rad_AP = [[moving_sum(rates_for_two_weeks_AP[:, i].reshape(2*7), j)[:7] 
                         for j in range(1, los+1)] for i in range(n_class)]
    # Convert to the right shape for later calculation
    rates_rad_AP = np.array(rates_rad_AP)
    rates_rad_AP = np.swapaxes(rates_rad_AP, 1, 2)

    # In current week, store rates for current week and next week
    wk1_rates_FCFS = wk2_rates_FCFS
    wk2_rates_FCFS = rates_update_FCFS
    rates_for_two_weeks_FCFS = np.concatenate((wk1_rates_FCFS, wk2_rates_FCFS)).reshape(2, n_class, 7)
    # Derive rates for rate class of r, arrival day of d and length of stay of d
    rates_rad_FCFS = [[moving_sum(rates_for_two_weeks_FCFS[:, i].reshape(2*7), j)[:7] 
                       for j in range(1, los+1)] for i in range(n_class)]
    # Convert to the right shape for later calculation
    rates_rad_FCFS = np.array(rates_rad_FCFS)
    rates_rad_FCFS = np.swapaxes(rates_rad_FCFS, 1, 2)

    ################################################################################
    ################################################################################
    ################################################################################

    # Consider second week stay nights, our week starts on Sunday, not Monday
    # Initialize rates for adaptive pricing algorithm and first-come, first-serve approach
    rates_DP = rates_DP
    rates_AP = rates_rad_AP
    rates_FCFS = rates_rad_FCFS
    # Order of rates as dynamic pricing rates, adaptive pricing rates, and FCFS rates
    ratesAll = [rates_DP, rates_AP, rates_FCFS]
    # Generate nonhomogenous Poisson process true bookings (i.e., customer demand) on a weekly basis
    # the mean of the Poisson process will be equal to the linear demand curve function of the room rates
    mus = [np.maximum(intercepts + slopes * rates, 0) for rates in ratesAll]
    demands_next = [[poisson.rvs(mu, size=1) for mu in np.nditer(mus_each)] for mus_each in mus]
    demands_next = [np.array(x).reshape(n_class, 7, los) for x in demands_next]

    demands_new_FCFS = demands_next[2] # some demands might be zero

    ################################################################################
    ################################################################################
    ################################################################################

    # Week 2 sell information
    nightlyRev_next = []
    wk2SellInfo = []
    wk2RoomSold_AP = []
    for i in range(7):
        buckets_night = buckets[i]
        # Three different algorithms, three different scenarios to consider
        capacity_left = [capacity] * 3
        # Info about the booking class type, rooms sold and revenues (single night revenue)
        soldAll = []
        roomSold_AP = []
        # Reverse the buckets for each night under the assumption that low ranked booking
        # classes arrive first
        pl_idx = 0
        for rads in reversed(buckets_night):
            sold_bucket = []
            sold_bucket_AP = 0
            for rad in rads:
                # For stay night revenue calculation, we only use one-night stay revenue, not
                # multiple stay night total revenue
                rad_single = (rad[0], i, 0)
                # Rooms sold equals smaller of demand and capacity left
                # Dynamic pricing
                sold_DP = min(demands_next[0][rad], capacity_left[0])
                # Adaptive pricing, the selling amount is constrained by the protection levels for higher classes
                try:
                    BL_AP = max(capacity_left[1]-list(reversed(thetas_old[i]))[pl_idx], 0)
                except IndexError:
                    BL_AP = capacity_left[1]

                sold_AP = min(demands_next[1][rad], BL_AP)
                sold_bucket_AP += sold_AP
                # First-come, first-serve
                sold_FCFS = min(demands_next[2][rad], capacity_left[2])

                sold = [sold_DP, sold_AP, sold_FCFS]

                rev = [ratesAll[i][rad_single] * sold[i] for i in range(len(demands_next))]
                sold_bucket.append((rad, sold, rev))
                # Update remaining capacity for the next virtual class
                capacity_left = [capacity_left[i]-sold[i] for i in range(len(sold))]
            soldAll.append(sold_bucket)
            roomSold_AP.append(sold_bucket_AP)
            pl_idx += 1

        # Remove empty lists
        soldAll = list(filter(None, soldAll))
        soldAll = list(itertools.chain.from_iterable(soldAll))
        wk2SellInfo.append(soldAll)
        wk2RoomSold_AP.append(roomSold_AP)

        # Extract revenue information and store it in revenue array
        revenue = [soldAll[i][2] for i in range(len(soldAll))]
        nightlyRev_next.append(revenue)

        # Calculate weekly revenue for each algorithm
    nightlyRev_next = [np.array(x) for x in nightlyRev_next]
    revSum = [np.sum(x, axis=0) for x in nightlyRev_next]
    wk2Rev = np.sum(revSum, axis=0)
    
    weeklyRev.append(wk2Rev)

    ################################################################################
    ################################################################################
    ################################################################################

    # Use adaptive pricing algoritm to derive new rates for week 3
    wk2RoomSold_AP = [list(reversed(wk2RoomSold_AP[i])) for i in range(len(wk2RoomSold_AP))]
    wk2RoomSold_AP = [np.array(x) for x in wk2RoomSold_AP]

    # Update protection levels according to the sales info
    roomSold_cumsum = [np.cumsum(x) for x in wk2RoomSold_AP]

    ################################################################################
    ################################################################################
    ################################################################################

    # Compute if the demand for a class exceeds its corresponding protection levels
    Y = [roomSold_cumsum[i][:-1] >= thetas_old[i] for i in range(7)]
    # Implement Equation(2) in vanRyzin-McGill 2000
    Z = [np.cumproduct(Y[i]) for i in range(7)]
    # Calculate H(theta, x)
    H = [ratios[i][1:] - Z[i] for i in range(7)]
    thetas_new = [thetas_old[i] - (param1/(param2+week)) * H[i] for i in range(7)]
    # Truncate at zero and sort it-- nonnegativity of protection levels 
    thetas_new = [np.minimum(np.maximum(thetas_new[i], 0), capacity) for i in range(7)]
    thetas_new = [sorted(thetas_new[i]) for i in range(7)]
    # Round to integers
    thetas_new = [np.round(thetas_new[i], 0) for i in range(7)]

    ################################################################################
    ################################################################################
    ################################################################################

    # Create a dummy booking class 1 so that we can find the partitioned protection levels
    # from nested ones in an easy way
    thetas_old_full = [np.concatenate(([0], thetas_old[i], [capacity])) for i in range(7)]
    thetas_new_full = [np.concatenate(([0], thetas_new[i], [capacity])) for i in range(7)]

    # Calculate partitioned protection level changes for each bucket in each night
    thetas_old_full_prt = [np.diff(thetas_old_full[i]) for i in range(7)]
    thetas_new_full_prt = [np.diff(thetas_new_full[i]) for i in range(7)]

    # Percent change for partitioned protection levels
    # When divide by zero, we assume the change is 1, or 100%.
    thetas_adj = [np.divide(thetas_new_full_prt[i], thetas_old_full_prt[i], 
                            out=(np.zeros_like(thetas_new_full_prt[i])+2),
                           where=thetas_old_full_prt[i]!=0) - 1 for i in range(7)]

    ################################################################################
    ################################################################################
    ################################################################################

    bkClass_bkt_uniq = {}
    rates_adj = {}
    for x, y in bkClass_bkt:
        if x in bkClass_bkt_uniq:
            bkClass_bkt_uniq[x].append((y))
            rates_adj[x].append((thetas_adj[y[0]][y[1]]))
        else:
            bkClass_bkt_uniq[x] = [(y)]
            rates_adj[x] = [(thetas_adj[y[0]][y[1]])]

    ################################################################################
    ################################################################################
    ################################################################################

    # Derive average changes for a booking class (i.e., r, a, d combination)
    rates_adj_avg = {}
    for k, v in rates_adj.items():
        avg_adj = np.round(np.mean(np.array(v)), 4)
        rates_adj_avg[k] = [(avg_adj)]
        single_rate = np.round((rates_rad_AP[k] * (1+rates_adj_avg[k][0])) / (k[2] + 1), 0)
        rates_adj_avg[k].append((single_rate))

    ################################################################################
    ################################################################################
    ################################################################################

    # Derive single stay night revenue for each rate class for each stay night of the week
    rate0_new = []
    rate1_new = []
    for bkt in buckets:
        # Flatten the bucket elements for each day of the week
        bkt_ls = list(itertools.chain.from_iterable(bkt))
        # For each booking class in the falttened list, extract the rates generated by the algorithm
        myRate0_dict = {my_key: rates_adj_avg[my_key] for my_key in bkt_ls if my_key[0] == 0}
        myRate1_dict = {my_key: rates_adj_avg[my_key] for my_key in bkt_ls if my_key[0] == 1}
        # Store it as an array
        myRate0 = np.array(list(myRate0_dict.values()))
        myRate1 = np.array(list(myRate1_dict.values()))
        # Calculate stay night rates for each rate class for each stay night
        myRate0_avg = np.round(np.mean(myRate0, axis=0)[1], 0)
        myRate1_avg = np.round(np.mean(myRate1, axis=0)[1], 0)
        # Create list for new rates to use for next week
        rate0_new.append(myRate0_avg)
        rate1_new.append(myRate1_avg)

    # New single night rates for next week
    rates_update_AP = np.concatenate((rate0_new, rate1_new)).reshape(n_class, 7)
    rates_update_AP = np.minimum(rates_update_AP, 2*rates_init)
    
    print(rates_update_AP)

    ################################################################################
    ################################################################################
    ################################################################################

    # Compute total demand by rate class for each stay night, which is then 
    # used to calculate demand change ratio for FCFS strategy
    demands_old_bynight_FCFS = [sum_of_elements(demands_old_FCFS[tuple(zip(*stay_index[i]))]) 
                                for i in range(7)]
    demands_new_bynight_FCFS = [sum_of_elements(demands_new_FCFS[tuple(zip(*stay_index[i]))]) 
                                for i in range(7)]
    
    demands_old_bynight_FCFS = np.array(demands_old_bynight_FCFS)
    demands_new_bynight_FCFS = np.array(demands_new_bynight_FCFS)
    with np.errstate(divide='ignore'):
        demands_adj_ratio = (demands_new_bynight_FCFS - demands_old_bynight_FCFS) / demands_old_bynight_FCFS
    demands_adj_ratio[np.isnan(demands_adj_ratio)] = 1

    demands_adj_ratio = np.swapaxes(demands_adj_ratio, 0, 1)
    demands_adj_ratio = np.maximum(-1, np.minimum(demands_adj_ratio, 1))
    rates_update_FCFS = wk1_rates_FCFS * (1 + demands_adj_ratio)

[[136. 172. 136. 130. 126. 125. 143.]
 [129. 203. 213. 174. 146. 140. 136.]]
[[139. 170. 139. 137. 131. 128. 146.]
 [141. 175. 174. 153. 140. 145. 138.]]
[[120. 126. 121. 116. 114. 133. 123.]
 [146. 196. 230. 230. 194. 168. 140.]]
[[159. 170. 145. 135. 115. 126. 160.]
 [133. 180. 181. 189. 174. 155. 155.]]
[[ 85. 100. 123. 128. 133. 131. 103.]
 [160. 190. 211. 201. 185. 173. 152.]]
[[178. 182. 156. 143. 138. 142. 157.]
 [177. 199. 183. 177. 160. 152. 179.]]
[[ 91.  82. 114. 121. 123. 133. 116.]
 [150. 157. 196. 192. 200. 184. 155.]]
[[170. 183. 156. 146. 148. 142. 146.]
 [182. 213. 194. 196. 166. 155. 182.]]
[[ 85.  70. 111. 135. 142. 148. 116.]
 [139. 126. 174. 178. 178. 184. 159.]]
[[191. 193. 151. 132. 128. 139. 154.]
 [197. 230. 216. 214. 194. 171. 178.]]
[[ 80.  72. 119. 134. 141. 137. 106.]
 [158. 130. 157. 172. 189. 184. 184.]]
[[202. 206. 160. 138. 142. 149. 154.]
 [230. 230. 230. 226. 192. 184. 184.]]
[[ 96.  74. 122. 139. 147. 149. 118.]
 [178. 131. 149. 160. 172. 184. 184.]]

In [34]:
weeklyRev

[array([91170., 74064., 80676.]),
 array([83735., 75971., 80420.]),
 array([89472.  , 69069.  , 87485.49]),
 array([90104.  , 77799.  , 76198.45]),
 array([91064. , 74158. , 75842.1]),
 array([91874.  , 80544.  , 67215.36]),
 array([88549.  , 79187.  , 68413.45]),
 array([91439.  , 71891.  , 42587.01]),
 array([89910.  , 75186.  , 63971.93]),
 array([90452. , 73277. , 63836.1]),
 array([91717.  , 69532.  , 16266.12]),
 array([87505.  , 64320.  , 88125.15]),
 array([90991.  , 70738.  , 25585.03]),
 array([89284.  , 73660.  , 43299.17]),
 array([80579.  , 67980.  , 42472.99]),
 array([89043. , 71332. , 31329.5]),
 array([90305.  , 68140.  , 43714.67]),
 array([91885. , 63971. , 28488.7]),
 array([90833.  , 74907.  , 53677.63]),
 array([87531.  , 57042.  , 23417.36]),
 array([88959.  , 71344.  , 46772.01]),
 array([89544. , 64577. , 25632.8]),
 array([91300.  , 60614.  , 36971.31]),
 array([86452.  , 58714.  , 33100.45]),
 array([91599.  , 62000.  , 34269.97]),
 array([88044.  , 64968.  ,

In [31]:
# Consider second week stay nights, our week starts on Sunday, not Monday
# Initialize rates for adaptive pricing algorithm and first-come, first-serve approach
rates_DP = rates_DP
rates_AP = rates_rad_old_AP
rates_FCFS = rates_rad_old_FCFS
# Order of rates as dynamic pricing rates, adaptive pricing rates, and FCFS rates
ratesAll = [rates_DP, rates_AP, rates_FCFS]

# Generate nonhomogenous Poisson process true bookings (i.e., customer demand) on a weekly basis
# the mean of the Poisson process will be equal to the linear demand curve function of the room rates
mus = [np.maximum(intercepts + slopes * rates, 0) for rates in ratesAll]
demands_next = [[poisson.rvs(mu, size=1) for mu in np.nditer(mus_each)] for mus_each in mus]
demands_next = [np.array(x).reshape(n_class, 7, los) for x in demands_next]

demands_new_FCFS = demands_next[2]

In [32]:
# Week 2 sell information
nightlyRev_next = []
wk2SellInfo = []
wk2RoomSold_AP = []
for i in range(7):
    buckets_night = buckets[i]
    # Three different algorithms, three different scenarios to consider
    capacity_left = [capacity] * 3
    # Info about the booking class type, rooms sold and revenues (single night revenue)
    soldAll = []
    roomSold_AP = []
    # Reverse the buckets for each night under the assumption that low ranked booking
    # classes arrive first
    pl_idx = 0
    for rads in reversed(buckets_night):
        sold_bucket = []
        sold_bucket_AP = 0
        for rad in rads:
            # For stay night revenue calculation, we only use one-night stay revenue, not
            # multiple stay night total revenue
            rad_single = (rad[0], i, 0)
            # Rooms sold equals smaller of demand and capacity left
            # Dynamic pricing
            sold_DP = min(demands_next[0][rad], capacity_left[0])
            # Adaptive pricing, the selling amount is constrained by the protection levels for higher classes
            try:
                BL_AP = max(capacity_left[1]-list(reversed(thetas_old[i]))[pl_idx], 0)
            except IndexError:
                BL_AP = capacity_left[1]
            
            sold_AP = min(demands_next[1][rad], BL_AP)
            sold_bucket_AP += sold_AP
            # First-come, first-serve
            sold_FCFS = min(demands_next[2][rad], capacity_left[2])
            
            sold = [sold_DP, sold_AP, sold_FCFS]
            
            rev = [ratesAll[i][rad_single] * sold[i] for i in range(len(demands_next))]
            sold_bucket.append((rad, sold, rev))
            # Update remaining capacity for the next virtual class
            capacity_left = [capacity_left[i]-sold[i] for i in range(len(sold))]
        soldAll.append(sold_bucket)
        roomSold_AP.append(sold_bucket_AP)
        pl_idx += 1

    # Remove empty lists
    soldAll = list(filter(None, soldAll))
    soldAll = list(itertools.chain.from_iterable(soldAll))
    wk2SellInfo.append(soldAll)
    wk2RoomSold_AP.append(roomSold_AP)

    # Extract revenue information and store it in revenue array
    revenue = [soldAll[i][2] for i in range(len(soldAll))]
    nightlyRev_next.append(revenue)
    
    # Calculate weekly revenue for each algorithm
nightlyRev_next = [np.array(x) for x in nightlyRev_next]
revSum = [np.sum(x, axis=0) for x in nightlyRev_next]
wk2Rev = np.sum(revSum, axis=0)
wk2Rev

array([90008., 82850., 80916.])

In [33]:
# Use adaptive pricing algoritm to derive new rates for week 3
wk2RoomSold_AP = [list(reversed(wk2RoomSold_AP[i])) for i in range(len(wk2RoomSold_AP))]
wk2RoomSold_AP = [np.array(x) for x in wk2RoomSold_AP]

# Update protection levels according to the sales info
roomSold_cumsum = [np.cumsum(x) for x in wk2RoomSold_AP]

In [34]:
# Compute if the demand for a class exceeds its corresponding protection levels
Y = [roomSold_cumsum[i][:-1] >= thetas_old[i] for i in range(7)]
# Implement Equation(2) in vanRyzin-McGill 2000
Z = [np.cumproduct(Y[i]) for i in range(7)]
# Calculate H(theta, x)
H = [ratios[i][1:] - Z[i] for i in range(7)]
thetas_new = [thetas_old[i] - (param1/(param2+1)) * H[i] for i in range(7)]
# Truncate at zero and sort it-- nonnegativity of protection levels 
thetas_new = [np.minimum(np.maximum(thetas_new[i], 0), capacity) for i in range(7)]
thetas_new = [sorted(thetas_new[i]) for i in range(7)]
# Round to integers
thetas_new = [np.round(thetas_new[i], 0) for i in range(7)]
thetas_old, thetas_new

([array([ 0., 26., 90.]),
  array([ 0., 23., 59., 88.]),
  array([11., 14., 61., 74.]),
  array([23., 47., 60.]),
  array([11., 47., 60.]),
  array([ 0.,  9., 25., 52.]),
  array([ 0., 11., 37., 63.])],
 [array([ 4., 16., 80.]),
  array([ 4., 10., 46., 80.]),
  array([13., 19., 69., 82.]),
  array([28., 52., 66.]),
  array([16., 53., 66.]),
  array([ 2., 13., 14., 44.]),
  array([ 2., 17., 44., 72.])])

In [35]:
# Create a dummy booking class 1 so that we can find the partitioned protection levels
# from nested ones in an easy way
thetas_old_full = [np.concatenate(([0], thetas_old[i], [capacity])) for i in range(7)]
thetas_new_full = [np.concatenate(([0], thetas_new[i], [capacity])) for i in range(7)]

# Calculate partitioned protection level changes for each bucket in each night
thetas_old_full_prt = [np.diff(thetas_old_full[i]) for i in range(7)]
thetas_new_full_prt = [np.diff(thetas_new_full[i]) for i in range(7)]

# Percent change for partitioned protection levels
# When divide by zero, we assume the change is 1, or 100%.
thetas_adj = [np.divide(thetas_new_full_prt[i], thetas_old_full_prt[i], 
                        out=(np.zeros_like(thetas_new_full_prt[i])+2),
                       where=thetas_old_full_prt[i]!=0) - 1 for i in range(7)]

In [36]:
bkClass_bkt_uniq = {}
rates_adj = {}
for x, y in bkClass_bkt:
    if x in bkClass_bkt_uniq:
        bkClass_bkt_uniq[x].append((y))
        rates_adj[x].append((thetas_adj[y[0]][y[1]]))
    else:
        bkClass_bkt_uniq[x] = [(y)]
        rates_adj[x] = [(thetas_adj[y[0]][y[1]])]

In [37]:
# Derive average changes for a booking class (i.e., r, a, d combination)
rates_adj_avg = {}
for k, v in rates_adj.items():
    avg_adj = np.round(np.mean(np.array(v)), 4)
    rates_adj_avg[k] = [(avg_adj)]
    single_rate = np.round((rates_rad_old_AP[k] * (1+rates_adj_avg[k][0])) / (k[2] + 1), 0)
    rates_adj_avg[k].append((single_rate))

In [38]:
# Derive single stay night revenue for each rate class for each stay night of the week
rate0_new = []
rate1_new = []
for bkt in buckets:
    # Flatten the bucket elements for each day of the week
    bkt_ls = list(itertools.chain.from_iterable(bkt))
    # For each booking class in the falttened list, extract the rates generated by the algorithm
    myRate0_dict = {my_key: rates_adj_avg[my_key] for my_key in bkt_ls if my_key[0] == 0}
    myRate1_dict = {my_key: rates_adj_avg[my_key] for my_key in bkt_ls if my_key[0] == 1}
    # Store it as an array
    myRate0 = np.array(list(myRate0_dict.values()))
    myRate1 = np.array(list(myRate1_dict.values()))
    # Calculate stay night rates for each rate class for each stay night
    myRate0_avg = np.round(np.mean(myRate0, axis=0)[1], 0)
    myRate1_avg = np.round(np.mean(myRate1, axis=0)[1], 0)
    # Create list for new rates to use for next week
    rate0_new.append(myRate0_avg)
    rate1_new.append(myRate1_avg)

# New single night rates for next week
rates_update_AP = np.concatenate((rate0_new, rate1_new)).reshape(n_class, 7)

In [39]:
rates_update_AP

array([[142., 166., 139., 143., 133., 122., 132.],
       [130., 135., 112., 100.,  98., 106., 108.]])

In [40]:
# Compute total demand by rate class for each stay night, which is then 
# used to calculate demand change ratio for FCFS strategy
demands_old_bynight_FCFS = [sum_of_elements(demands_old_FCFS[tuple(zip(*stay_index[i]))]) 
                            for i in range(7)]
demands_new_bynight_FCFS = [sum_of_elements(demands_new_FCFS[tuple(zip(*stay_index[i]))]) 
                            for i in range(7)]

demands_adj_ratio = [[(demands_new_bynight_FCFS[i][j] - 
                       demands_old_bynight_FCFS[i][j]) / demands_old_bynight_FCFS[i][j] 
 for j in range(n_class)] for i in range(7)]
demands_adj_ratio = np.array(demands_adj_ratio)
demands_adj_ratio = np.swapaxes(demands_adj_ratio, 0, 1)
rates_update_FCFS = wk1_rates_FCFS * (1 + demands_adj_ratio)
rates_update_FCFS

array([[122.57, 140.7 , 162.  , 131.49, 140.7 , 101.92, 116.2 ],
       [ 82.74,  96.93, 119.54, 147.61, 113.49,  84.44,  89.61]])